In [1]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# Каталог с данными для обучения
train_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\train'
# Каталог с данными для проверки
val_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\validation'
# Каталог с данными для тестирования
test_dir = r'C:\Users\Dmitry\Desktop\CNN_Research_2018\cats_vs_dogs\test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width, img_height, 3)
# Размер мини-выборки
batch_size = 8
# Количество изображений для обучения
nb_train_samples = 17500
# Количество изображений для проверки
nb_validation_samples = 3750
# Количество изображений для тестирования
nb_test_samples = 3750

In [3]:
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

In [4]:
vgg16_net.trainable = False

In [14]:
vgg16_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [5]:
model = Sequential()
# Добавляем в модель сеть VGG16 вместо слоя
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_2 (Activation)    (None, 1)                 0         
Total para

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [9]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 17500 images belonging to 2 classes.


In [10]:

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [11]:

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 3750 images belonging to 2 classes.


In [12]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
2187/2187 [==============================] - 466s 213ms/step - loss: 0.3980 - acc: 0.8122 - val_loss: 0.2802 - val_acc: 0.8854
Epoch 2/10
2187/2187 [==============================] - 484s 221ms/step - loss: 0.2736 - acc: 0.8827 - val_loss: 0.2461 - val_acc: 0.9003
Epoch 3/10
2187/2187 [==============================] - 484s 221ms/step - loss: 0.2402 - acc: 0.8993 - val_loss: 0.2274 - val_acc: 0.9089
Epoch 4/10
2187/2187 [==============================] - 480s 219ms/step - loss: 0.2253 - acc: 0.9056 - val_loss: 0.2221 - val_acc: 0.9110
Epoch 5/10
2187/2187 [==============================] - 480s 219ms/step - loss: 0.2091 - acc: 0.9141 - val_loss: 0.2166 - val_acc: 0.9126
Epoch 6/10
2187/2187 [==============================] - 479s 219ms/step - loss: 0.1983 - acc: 0.9192 - val_loss: 0.2174 - val_acc: 0.9126
Epoch 7/10
2187/2187 [==============================] - 477s 218ms/step - loss: 0.1848 - acc: 0.9235 - val_loss: 0.2102 - val_acc: 0.9126
Epoch 8/10
2187/2187 [============

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))